<a href="https://colab.research.google.com/github/alibekk93/IDP_analysis/blob/RAPID/getting_proteomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np

In [ ]:
!pip install bio

In [7]:
from Bio import Entrez
from Bio import SeqIO

In [2]:
tempura = pd.read_csv('/content/200617_TEMPURA.csv')

In [38]:
tempura.columns

Index(['genus_and_species', 'taxonomy_id', 'strain', 'superkingdom', 'phylum',
       'class', 'order', 'family', 'genus', 'assembly_or_accession',
       'Genome_GC', 'Genome_size', '16S_accssion', '16S_GC', 'Tmin',
       'Topt_ave', 'Topt_low', 'Topt_high', 'Tmax', 'Tmax_Tmin'],
      dtype='object')

In [44]:
tempura.dropna(subset='assembly_or_accession', inplace=True)
tempura.reset_index(drop=True, inplace=True)

In [45]:
tempura

,genus_and_species,taxonomy_id,strain,superkingdom,phylum,class,order,family,genus,assembly_or_accession,Genome_GC,Genome_size,16S_accssion,16S_GC,Tmin,Topt_ave,Topt_low,Topt_high,Tmax,Tmax_Tmin
0,Pyrolobus fumarii,54252,1A,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,GCA_000223395.1,54.9,1.8,X99555,68.4,90.0,106.0,NaN,NaN,113.0,23.0
1,Pyrococcus kukulkanii,1609559,NCB100,Archaea,Euryarchaeota,Thermococci,Thermococcales,Thermococcaceae,Pyrococcus,GCA_001577775.1,44.6,2.0,CP010835,66.5,70.0,105.0,NaN,NaN,112.0,42.0
2,Methanopyrus kandleri,2320,AV19,Archaea,Euryarchaeota,Methanopyri,Methanopyrales,Methanopyraceae,Methanopyrus,GCA_000007185.1,61.2,1.7,NR_074539,68.1,84.0,98.0,NaN,NaN,110.0,38.0
3,Pyrococcus yayanosii,1008460,CH1,Archaea,Euryarchaeota,Thermococci,Thermococcales,Thermococcaceae,Pyrococcus,GCA_000215995.1,51.6,1.7,EU682399,66.7,80.0,98.0,NaN,NaN,108.0,28.0
4,Hyperthermus butylicus,54248,DSM 5456,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Hyperthermus,GCA_000015145.1,53.7,1.7,X99553,66.3,72.0,100.5,95.0,106.0,108.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,Psychroflexus torquis,57029,Psychroflexus torquis,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Psychroflexus,GCA_000153485.2,34.5,4.3,U85881,48.9,-16.0,12.5,10.0,15.0,19.0,35.0
1058,Photobacterium profundum,74109,DSJ4,Bacteria,Proteobacteria,Gammaproteobacteria,Vibrionales,Vibrionaceae,Photobacterium,GCA_000196255.1,41.7,6.4,D21226,53.8,4.0,10.0,8.0,12.0,18.0,14.0
1059,Paraglaciecola psychrophila,326544,170,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Alteromonadaceae,Paraglaciecola,GCA_000347635.1,40.4,5.4,DQ007436,52.1,4.0,12.0,NaN,NaN,15.0,11.0
1060,Shewanella benthica,43661,W 145,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,GCA_900476435.1,45.9,4.4,X82131,53.4,0.0,5.0,NaN,NaN,10.0,10.0


In [51]:
failures = []
species = []

for id in tempura.loc[10:30, 'taxonomy_id']:
  try:
    Entrez.email = 'arkug@uottawa.ca'
    taxonomy_id = id
    handle = Entrez.efetch(db="taxonomy", id=taxonomy_id, retmode="xml")
    record = Entrez.read(handle)
    organism_name = record[0]["ScientificName"].replace(' ', '_')
    species.append(organism_name)
    handle.close()
    handle = Entrez.esearch(db='protein', term=f'txid{taxonomy_id}[Organism]', retmax=100000)
    record = Entrez.read(handle)
    handle.close()
    id_list = record['IdList']
    handle = Entrez.efetch(db='protein', id=id_list, rettype='fasta', retmode='text')
    records = SeqIO.parse(handle, 'fasta')
    SeqIO.write(records, f'{organism_name}.fasta', 'fasta')
    handle.close()
  except:
    failures.append(id)
    pass

In [67]:
taxonomy_id = '526'
url = f'https://rest.uniprot.org/proteomes/search?compressed=true&format=list&query=%28%28taxonomy_id%3A1609559%29%29&size=5'
response = requests.get(url)

In [70]:
response.content

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x00\x0b\r0\x00\x02s\x03S\x0bs.\x00\x90\xf8\xde\xe9\x0c\x00\x00\x00'

In [68]:
with open(f"{taxonomy_id}.txt", "wb") as f:
    f.write(response.content)

In [54]:
import requests
import gzip

taxonomy_id = "562"  # Replace with your taxonomy ID
url = f"https://www.uniprot.org/uniprot/?query=taxonomy:{taxonomy_id}&format=fasta&compress=yes"
response = requests.get(url)
with open(f"{taxonomy_id}.fasta.gz", "wb") as f:
    f.write(response.content)

with gzip.open(f"{taxonomy_id}.fasta.gz", "rt") as f:
    content = f.read()

BadGzipFile: ignored

In [52]:
failures

[2265, 1104324, 985052, 229980, 2180, 1273541, 94694, 2271, 187880, 187878]

In [50]:
!zip -r /content/fasta_files.zip /content -i '*.fasta'
from google.colab import files
files.download('/content/fasta_files.zip')

  adding: content/Pyrobaculum_aerophilum.fasta (deflated 53%)
  adding: content/Pyrococcus_furiosus.fasta (deflated 54%)
  adding: content/Pyrolobus_fumarii.fasta (deflated 48%)
  adding: content/Pyrococcus_yayanosii.fasta (deflated 48%)
  adding: content/Pyrobaculum_islandicum.fasta (deflated 50%)
  adding: content/Pyrococcus_abyssi.fasta (deflated 51%)
  adding: content/Hyperthermus_butylicus.fasta (deflated 49%)
  adding: content/Pyrococcus_horikoshii.fasta (deflated 59%)
  adding: content/Pyrococcus_kukulkanii.fasta (deflated 48%)
  adding: content/Escherichia_coli.fasta (deflated 45%)
  adding: content/Thermoproteus_uzoniensis.fasta (deflated 50%)
  adding: content/Methanopyrus_kandleri.fasta (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>